In [39]:
from priors import *
from model_configs import get_prior_config, create_configspace_from_hierarchical, sample_differentiable, evaluate_hypers
from priors.mlp import get_batch
from scripts import model_builder
import math
import random
import time
import warnings
from datetime import datetime

import torch

import numpy as np

import matplotlib.pyplot as plt
#from scripts.differentiable_pfn_evaluation import eval_model_range
# No string
from scripts.model_builder import get_model, get_default_spec, save_model, load_model
from scripts.transformer_prediction_interface import transformer_predict, get_params_from_config, load_model_workflow

from scripts.model_configs import *

from datasets import load_openml_list, open_cc_dids, open_cc_valid_dids
from priors.utils import plot_prior, plot_features
from priors.utils import uniform_int_sampler_f

from scripts.tabular_metrics import calculate_score_per_method, calculate_score
# No string
#from scripts.tabular_evaluation import evaluate

from priors.differentiable_prior import DifferentiableHyperparameterList, draw_random_style, merge_style_with_info, replace_differentiable_distributions
from scripts import tabular_metrics
from notebook_utils import *

In [52]:
# Setting up the configuration for the simulation of batch
config = get_prior_config("causal")

#hyper1 = model_builder.get_mlp_prior_hyperparameters(config)
hyper1 = evaluate_hypers(config, sample_diff_hps=True)
hyper = sample_differentiable(hyper1)

print(hyper)

{'lr': 0.00011384360782765453, 'dropout': 0.0, 'emsize': 256, 'batch_size': 64, 'nlayers': 12, 'num_features': 100, 'nhead': 4, 'nhid_factor': 2, 'bptt': 50, 'eval_positions': None, 'seq_len_used': 50, 'sampling': 'mixed', 'epochs': 80, 'num_steps': 100, 'verbose': False, 'mix_activations': False, 'pre_sample_causes': True, 'multiclass_type': 'value', 'nan_prob_unknown_reason_reason_prior': 0.5, 'categorical_feature_p': 0.1, 'nan_prob_no_reason': 0.0, 'nan_prob_unknown_reason': 0.0, 'nan_prob_a_reason': 0.0, 'max_num_classes': 2, 'num_classes': 2, 'noise_type': 'Gaussian', 'balanced': True, 'normalize_to_ranking': False, 'set_value_to_nan': 0.2, 'normalize_by_used_features': True, 'num_features_used': {'uniform_int_sampler_f(3,max_features)': <function <lambda>.<locals>.<lambda> at 0x7f9790314160>}, 'num_categorical_features_sampler_a': -1.0, 'prior_bag_exp_weights_1': 4.139472938986653, 'num_layers': <function DifferentiableHyperparameter.__init__.<locals>.make_gamma.<locals>.<lambda>

In [12]:
# Setting up the configuration for the simulation of batch
config = get_prior_config("causal")
#config = model_builder.get_mlp_prior_hyperparameters(config)
print(config)
# config = evaluate_hypers(config, sample_diff_hps=True)
config = sample_differentiable(config)
print(hyper)
model_builder.get_model(config, device = 'cpu', should_train=True, verbose=True, state_dict=None, epoch_callback=None)

{'lr': lr, Type: UniformFloat, Range: [0.0001, 0.00015], Default: 0.0001224745, on log-scale, 'dropout': dropout, Type: Categorical, Choices: {0.0}, Default: 0.0, 'emsize': emsize, Type: Categorical, Choices: {256}, Default: 256, 'batch_size': batch_size, Type: Categorical, Choices: {64, 128}, Default: 64, 'nlayers': nlayers, Type: Categorical, Choices: {12}, Default: 12, 'num_features': 100, 'nhead': nhead, Type: Categorical, Choices: {4}, Default: 4, 'nhid_factor': 2, 'bptt': 50, 'eval_positions': None, 'seq_len_used': 50, 'sampling': 'normal', 'epochs': 80, 'num_steps': 100, 'verbose': False, 'mix_activations': False, 'pre_sample_causes': True, 'multiclass_type': 'rank', 'nan_prob_unknown_reason_reason_prior': nan_prob_unknown_reason_reason_prior, Type: Categorical, Choices: {0.5}, Default: 0.5, 'categorical_feature_p': categorical_feature_p, Type: Categorical, Choices: {0.0, 0.1, 0.2}, Default: 0.0, 'nan_prob_no_reason': nan_prob_no_reason, Type: Categorical, Choices: {0.0, 0.1}, D

TypeError: unsupported operand type(s) for *: 'ConfigSpace.hyperparameters.CategoricalHyperparameter' and 'ConfigSpace.hyperparameters.CategoricalHyperparameter'

In [3]:
# hyper['num_layers'] = hyper['num_layers']()
# hyper['prior_mlp_hidden_dim'] = hyper['prior_mlp_hidden_dim']()
# hyper['prior_mlp_dropout_prob'] = hyper['prior_mlp_dropout_prob']()
# hyper['noise_std'] = hyper['noise_std']()
# hyper['init_std'] = hyper['init_std']()
# hyper['num_causes'] = hyper['num_causes']()
# hyper['prior_mlp_dropout_prob'] = hyper['prior_mlp_dropout_prob']()


In [51]:
batch = get_batch(batch_size = 1000, seq_len = 100, num_features = 100, hyperparameters = hyper)

TypeError: unsupported operand type(s) for -: 'function' and 'int'

In [71]:
def reload_config(config_type='causal', task_type='multiclass', longer=0):
    config = get_prior_config(config_type=config_type)
    
    config['prior_type'], config['differentiable'], config['flexible'] = 'mlp', True, True
    
    model_string = ''
    
    config['epochs'] = 12000
    config['recompute_attn'] = True

    config['max_num_classes'] = 10
    config['num_classes'] = uniform_int_sampler_f(2, config['max_num_classes'])
    config['balanced'] = False
    model_string = model_string + '_multiclass'
    
    model_string = model_string + '_'+datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    
    return config, model_string

In [82]:
config, model_string = reload_config(longer=1)

config['bptt_extra_samples'] = None

# diff
config['output_multiclass_ordered_p'] = 0.
del config['differentiable_hyperparameters']['output_multiclass_ordered_p']

config['multiclass_type'] = 'rank'
del config['differentiable_hyperparameters']['multiclass_type']

config['sampling'] = 'normal' # vielleicht schlecht?
del config['differentiable_hyperparameters']['sampling']

config['pre_sample_causes'] = True
# end diff

config['multiclass_loss_type'] = 'nono' # 'compatible'
config['normalize_to_ranking'] = False # False

config['categorical_feature_p'] = .2 # diff: .0

# turn this back on in a random search!?
config['nan_prob_no_reason'] = .0
config['nan_prob_unknown_reason'] = .0 # diff: .0
config['set_value_to_nan'] = .1 # diff: 1.

config['normalize_with_sqrt'] = False

config['new_mlp_per_example'] = True
config['prior_mlp_scale_weights_sqrt'] = True
config['batch_size_per_gp_sample'] = None

config['normalize_ignore_label_too'] = False

config['differentiable_hps_as_style'] = False
config['max_eval_pos'] = 1000

config['random_feature_rotation'] = True
config['rotate_normalized_labels'] = True

config["mix_activations"] = False # False heisst eig True

config['emsize'] = 512
config['nhead'] = config['emsize'] // 128
config['bptt'] = 1024+128
config['canonical_y_encoder'] = False

    
config['aggregate_k_gradients'] = 8
config['batch_size'] = 8*config['aggregate_k_gradients']
config['num_steps'] = 1024//config['aggregate_k_gradients']
config['epochs'] = 400
config['total_available_time_in_s'] = None #60*60*22 # 22 hours for some safety...

config['train_mixed_precision'] = True
config['efficient_eval_masking'] = True

config = evaluate_hypers(config)
config['batch_size'] = 4
print(config)

{'lr': 0.00010443982527019839, 'dropout': 0.0, 'emsize': 512, 'batch_size': 4, 'nlayers': 12, 'num_features': 100, 'nhead': 4, 'nhid_factor': 2, 'bptt': 1152, 'eval_positions': None, 'seq_len_used': 50, 'sampling': 'normal', 'epochs': 400, 'num_steps': 128, 'verbose': False, 'mix_activations': False, 'pre_sample_causes': True, 'multiclass_type': 'rank', 'nan_prob_unknown_reason_reason_prior': 0.5, 'categorical_feature_p': 0.2, 'nan_prob_no_reason': 0.0, 'nan_prob_unknown_reason': 0.0, 'nan_prob_a_reason': 0.0, 'max_num_classes': 10, 'num_classes': <function <lambda>.<locals>.<lambda> at 0x7f8e202e5280>, 'noise_type': 'Gaussian', 'balanced': False, 'normalize_to_ranking': False, 'set_value_to_nan': 0.1, 'normalize_by_used_features': True, 'num_features_used': {'uniform_int_sampler_f(3,max_features)': <function <lambda>.<locals>.<lambda> at 0x7f8e202e51f0>}, 'num_categorical_features_sampler_a': -1.0, 'differentiable_hyperparameters': {'prior_bag_exp_weights_1': {'distribution': 'uniform

In [73]:
def make_get_batch(model_proto, **extra_kwargs):
    def new_get_batch(batch_size, seq_len, num_features, hyperparameters
            , device, model_proto=model_proto
            , **kwargs):
        kwargs = {**extra_kwargs, **kwargs} # new args overwrite pre-specified args
        return model_proto.get_batch(
            batch_size=batch_size
            , seq_len=seq_len
            , device=device
            , hyperparameters=hyperparameters
            , num_features=num_features, **kwargs)
    return new_get_batch

In [74]:
def eval_pos_seq_len_sampler():
    single_eval_pos = single_eval_pos_gen()
    if bptt_extra_samples:
        return single_eval_pos, single_eval_pos + bptt_extra_samples
    else:
        return single_eval_pos, bptt

In [84]:

args = replace_differentiable_distributions(config)


print(config)

{'lr': 0.00010443982527019839, 'dropout': 0.0, 'emsize': 512, 'batch_size': 4, 'nlayers': 12, 'num_features': 100, 'nhead': 4, 'nhid_factor': 2, 'bptt': 1152, 'eval_positions': None, 'seq_len_used': 50, 'sampling': 'normal', 'epochs': 400, 'num_steps': 128, 'verbose': False, 'mix_activations': False, 'pre_sample_causes': True, 'multiclass_type': 'rank', 'nan_prob_unknown_reason_reason_prior': 0.5, 'categorical_feature_p': 0.2, 'nan_prob_no_reason': 0.0, 'nan_prob_unknown_reason': 0.0, 'nan_prob_a_reason': 0.0, 'max_num_classes': 10, 'num_classes': <function <lambda>.<locals>.<lambda> at 0x7f8e202e5280>, 'noise_type': 'Gaussian', 'balanced': False, 'normalize_to_ranking': False, 'set_value_to_nan': 0.1, 'normalize_by_used_features': True, 'num_features_used': {'uniform_int_sampler_f(3,max_features)': <function <lambda>.<locals>.<lambda> at 0x7f8e202e51f0>}, 'num_categorical_features_sampler_a': -1.0, 'differentiable_hyperparameters': {'prior_bag_exp_weights_1': {'distribution': 'uniform

In [85]:
if 'aggregate_k_gradients' not in config or config['aggregate_k_gradients'] is None:
    config['aggregate_k_gradients'] = math.ceil(config['batch_size'] * ((config['nlayers'] * config['emsize'] * config['bptt'] * config['bptt']) / 10824640000))

config['num_steps'] = math.ceil(config['num_steps'] * config['aggregate_k_gradients'])
config['batch_size'] = math.ceil(config['batch_size'] / config['aggregate_k_gradients'])
config['recompute_attn'] = config['recompute_attn'] if 'recompute_attn' in config else False
single_eval_pos_gen =  lambda: 100, 150



kwargs = model_builder.get_mlp_prior_hyperparameters(config)

print(kwargs)
extra_prior_kwargs_dict={
            'num_steps': config['num_steps'],
            'num_features': config['num_features']
            , 'hyperparameters': kwargs
            #, 'dynamic_batch_size': 1 if ('num_global_att_tokens' in config and config['num_global_att_tokens']) else 2
            , 'batch_size_per_gp_sample': config.get('batch_size_per_gp_sample', None)
        }

def sampler():
    return 100, 150
mlp.DataLoader(batch_size = 1000, eval_pos_seq_len_sampler = sampler, **extra_prior_kwargs_dict).get_test_batch()


{'lr': 0.00010443982527019839, 'dropout': 0.0, 'emsize': 512, 'batch_size': 1, 'nlayers': 12, 'num_features': 100, 'nhead': 4, 'nhid_factor': 2, 'bptt': 1152, 'eval_positions': None, 'seq_len_used': 50, 'sampling': 'normal', 'epochs': 400, 'num_steps': 1024, 'verbose': False, 'mix_activations': False, 'pre_sample_causes': True, 'multiclass_type': 'rank', 'nan_prob_unknown_reason_reason_prior': 0.5, 'categorical_feature_p': 0.2, 'nan_prob_no_reason': 0.0, 'nan_prob_unknown_reason': 0.0, 'nan_prob_a_reason': 0.0, 'max_num_classes': 10, 'num_classes': <function <lambda>.<locals>.<lambda> at 0x7f8e202e5280>, 'noise_type': 'Gaussian', 'balanced': False, 'normalize_to_ranking': False, 'set_value_to_nan': 0.1, 'normalize_by_used_features': True, 'num_features_used': <function <lambda>.<locals>.<lambda> at 0x7f8e202e51f0>, 'num_categorical_features_sampler_a': -1.0, 'differentiable_hyperparameters': {'distribution': 'uniform', 'min': 2.0, 'max': 10.0, 'sample': prior_bag_exp_weights_1, Type: U

KeyError: 'prior_mlp_activations'